In [124]:
from hypergraphs import *
from kegg import *
import pandas as pd
import os
import logging
from Bio.KEGG.KGML.KGML_parser import read
from ast import literal_eval
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import requests
from Levenshtein import ratio
import scipy.stats

In [4]:
organismDict = {"hsa": "Homo sapiens (human)"}
outputDirectory = "/Users/boldi/Desktop/pw/"
inputDirectory = "../../LaTeX/Data/KEGG-Pathways/Nov2023/"
dataDirectory = "../../LaTeX/Data/KEGG-Pathways"

In [5]:
pws = pd.read_csv(os.path.join(inputDirectory, "KEGG-pathways.txt"), header=0, 
            names=["Pathway ID", "Pathway type", "Pathway name"], dtype="string", keep_default_na=False)


In [6]:
superpathwayDict = {
    "Involved": [(x,y) for x,y in zip(pws["Pathway ID"], pws["Pathway name"])]
}

In [7]:
superpathway = "Involved"
organism = "hsa"

In [8]:
safeSuperpathway = makesafe(superpathway)

In [9]:
def words2set(words):
    """
        Given a space-separated list of words, returns it as a set of words
    """
    return set(words.split())

In [10]:
from typing import NamedTuple

class Relation(NamedTuple):
    """
        Represents a relation. It is characterized by:
        - a type (e.g. "PPRel") 
        - a list of subtypes (e.g. [('activation', '-->'), ('indirect effect', '..>')])
        - a set of components (source)
        - a list of sets of components (target)
    """
    relType: str
    relSubtypes: list
    source: set
    target: list

    @classmethod
    def fromKEGG(cls, relation, pathway):
        """
            Constructor: builds a Relation from a KEGG relation
        """
        if relation.entry2.name == "undefined":
            s = []
            for component in relation.entry2.components:
                s += [words2set(pathway.entries[component.id].name)]
        else:
            s = [words2set(relation.entry2.name)]
        return cls(relation.type, relation.subtypes, words2set(relation.entry1.name), s)
        

In [11]:
def relationsFromKEGGpathway(pathway):
    """
        Given a KEGG pathway, returns the list of relations it contains (represented as Relation).
    """
    return [Relation.fromKEGG(r, pathway) for r in pathway.relations]
    

In [52]:
def pathwayFromFile(dataDirectory, organism, pathwayID):
    return KGML_parser.read(open(os.path.join(dataDirectory, organism, organism+pathwayID+".xml"), "r"))

In [53]:
def relationsFromFile(dataDirectory, organism, pathwayID):
    """
        Given a pathway file (specified by a root dataDirectory, the name of the organism subdirectory, and the name
        of the file [organism+pathwayID.xml]), reads it and returns the list of its relations.
    """
    pathway = pathwayFromFile(dataDirectory, organism, pathwayID)
    return relationsFromKEGGpathway(pathway)

In [54]:
def relationsForSuperpathway(dataDirectory, organism, superpathwayDict, superpathway):
    """
        Given a superpathDict (whose keys are superpathway names and whose values are list of
        pairs (pathwayID, pathwayName)), reads all the pathway file for a specific superpathway name
        and returns a dictionary whose keys are the pathwayID's and whose values are the list of relations.
    """
    rel = {}
    for k,v in superpathwayDict[superpathway]:
        #logging.info("Reading", k, v)
        rel[k] = relationsFromFile(dataDirectory, organism, k)
    return rel

In [55]:
def subtypes(rel):
    """
        Given a map whose values are relations, accumulate all subtypes appearing and assign them a number.
        The result is a map from subtype string to number.
    """
    all_subtypes = set([])
    for pathwayid, relations in rel.items():
        for relation in relations:
            all_subtypes |= set([str(relation.relSubtypes)])
        
    s = sorted(list(all_subtypes))
    subtype2color = {v:k for k,v in enumerate(s)}

    return subtype2color

In [15]:
def set2can(s):
    """
        Convert a set to a string in a canonical way.
    """
    return str(sorted(list(s)))

def can2set(c):
    """
        Does the converse of set2can.
    """
    return set(literal_eval(c))

In [16]:
def indices(rel):
    """
        Given a map whose values are lists of relations, it considers all the relations one by one, and attributes a unique id 
        to each element (i.e., set of components appearing as source or in the target of some relation) and block (the set of
        element id appearing as target of some relation).
        This function returns the dictionaries to move from/to an element or block to the corresponding id.
        Elements are string representations of sorted lists of strings.
        Blocks are string representations of sorted lists of ints.
    """
    element2id = {}
    id2element = {}
    block2id = {}
    id2block = {}

    for relations in rel.values():
        for relation in relations:
            s = set2can(relation.source)
            if s not in element2id.keys():
                element2id[s] = len(element2id)
            targetset = set([])
            for targ in relation.target:
                t = set2can(targ)
                if t not in element2id.keys():
                    element2id[t] = len(element2id)
                targetset |= set([element2id[t]])
            ts = set2can(targetset)
            if ts not in block2id.keys():
                block2id[ts] = len(block2id)

    for k,v in element2id.items():
        id2element[v] = k
    for k,v in block2id.items():
        id2block[v] = k
    return element2id, id2element, block2id, id2block


In [17]:
def search_gene_KEGG(geneID):
    """
        Search for gene on KEGG. 
        
        Returns list of names.
    """
    url1 = "https://www.kegg.jp/entry/"+geneID
    response1 = requests.get(url1, allow_redirects=False)    
    soup = BeautifulSoup(response1.text, "html.parser")
    tds = [tds for tds in soup.find_all("td", {"class": "td11 defd"})]
    names = tds[0].getText().strip().split(", ")
    return names

In [18]:
rel = relationsForSuperpathway(dataDirectory, organism, superpathwayDict, superpathway)
element2id, id2element, block2id, id2block = indices(rel)
subtype2color = subtypes(rel)
elements = list(set.union(*[can2set(k) for k in element2id.keys()]))

In [19]:
# name2gene is a function mapping names to gene IDs (hsa:xxxx)
# e.g. KCNE3, BRGDA6 etc are all mapped to hsa:10008 (see https://www.kegg.jp/entry/hsa:10008)
# This map was produced by scraping kegg, but this has been done only once.
# After that, we just read a pkl file

name2keggFilename = os.path.join(inputDirectory, "name2kegg.pkl")

if os.path.exists(name2keggFilename):
    print("Reading gene names")
    with open(name2keggFilename, "rb") as handle:
        name2gene = pickle.load(handle)
else:
    print("Producing gene names")
    d = {}
    nelements = len(elements)
    for count,element in enumerate(elements):
        if count % 10 == 0:
            print(f"{count}/{nelements}")
        if element.startswith("hsa"):
            names = search_gene_KEGG(element)
            for name in names:
                d[name] = element
    with open(name2keggFilename, "wb") as handle:
        pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)
    name2gene = d

Reading gene names


In [20]:
# Building the graph of all relations contained in rel values
# Nodes are IDs, and node x correspond to the set of genes (and/or compound) id2element[x]
G = nx.DiGraph() 
for relations in rel.values():
    for relation in relations:
        source = element2id[set2can(relation.source)]
        for targ in relation.target:
            target = element2id[set2can(targ)]
            G.add_edge(source, target, color=subtype2color[str(relation.relSubtypes)])

In [21]:
from qf.cc import cardon_crochemore_colored, cardon_crochemore

cc=cardon_crochemore_colored(G)

In [58]:
pathway = pathwayFromFile(dataDirectory, organism, superpathwayDict[superpathway][0][0])
canvas = KGMLCanvas(pathway)


In [22]:
# Read patient data
genes = pd.read_csv(os.path.join(inputDirectory, "genes.csv"))
patients = genes["name"].values.tolist()
classification = pd.read_csv(os.path.join(inputDirectory, "classification.csv"))
patient2class = {patient: classification.loc[classification["name"]==patient]["classification"].values.flatten().tolist()[0] for patient in patients}

In [23]:
# Build node2columns, mapping each node of G to the list of genes corresponding to it
# for which we know a value (in the genes dataframe), sorted
#
namedGenes = set(name2gene.values())     
columns = set(genes.columns.values.tolist()[2:])
node2columns = {}
for node in G.nodes():
    nodeGenes = can2set(id2element[node])
    s = []
    for gene in nodeGenes & namedGenes:
        for k,v in name2gene.items():
            if v == gene:
                if k in columns:
                    s += [k]
    node2columns[node]=sorted(s)

In [24]:
def mapTranscrToG(G, node2columns, genes, patient):
    """
        Returns a map mapping each node of G to the list of values of genes corresponding to that node
        (according to node2columns) for the given patient
    """
    return {node: genes.loc[genes["name"] == patient][node2columns[node]].values.flatten().tolist() for node in G.nodes()}

In [25]:
import statistics
import random

def mapToAvg(d):
    """
        Given a map from the nodes of a graph to lists of floats, returns a map whose values are the averages
        (or nan if the list is empty)
    """
    res = {}
    for node in G.nodes():
        if len(d[node]) == 0:
            res[node] = np.nan
        else:
            res[node] = statistics.mean(d[node])
    return res

In [26]:
def sd(d, ks):
    """
        Given a map d whose values are floats or nan, and given a set of keys ks, returns the
        standard deviation of the non-nan values of the nodes in ks, and the corresponding size.
        Returns nan if all the values are nan, 0 if there is a single datapoint.
    """
    values = [d[k] for k in ks if not np.isnan(d[k])]
    if len(values) == 0:
        return (np.nan,0)
    if len(values) == 1:
        return (0,1)
    return (statistics.stdev(values), len(values))

In [27]:
def nsd(d, ks, rep=1000):
    """
        Given a map d (whose values are floats or nans) and a set of keys ks:
        1) compute sd(d, ks) (the standard deviation of the values corresponding to ks, ignoring nans)
        2) compare this with the sd of sets of keys of the same size as ks, but drawn at random
        The experiment (2) is repeated rep times and the non-nan results are averaged.
        The ratio between (1) and the average resulting from (2) is returned.
        Values larger/smaller than 1 mean "more overexpressed/underexpress than the average"
    """ 
    sigma, k = sd(d, ks)
    if sigma == 0:
        return 1
    v = 0
    keys = list(d.keys())
    for i in range(rep):
        smpl = random.sample(keys, len(ks))
        ss, kk = sd(d, smpl)
        #print(f"Trying with sample {smpl}, got {ss}")
        if not np.isnan(ss):
            v += ss
    sigma0 = v / i
    return sigma/sigma0

In [144]:
from collections import defaultdict 

healthy = defaultdict(list)
diseased = defaultdict(list)

for patient in patients:
    m=mapTranscrToG(G, node2columns, genes, patient)
    ma=mapToAvg(m)
    for k in set(cc.values()):
        fibre = [x for x in cc.keys() if cc[x]==k]
        if len(fibre) > 1:
            if patient2class[patient] == "healthy":
                healthy[k] +=[nsd(ma, fibre)]
            else:
                diseased[k] += [nsd(ma, fibre)]


In [157]:
pd.DataFrame(healthy[61]).describe()

,0
count,29.000000
mean,1.133259
std,0.341987
min,0.411102
25%,0.890853
50%,1.167061
75%,1.288276
max,1.822055


In [156]:
pd.DataFrame(diseased[61]).describe()

,0
count,35.000000
mean,1.093326
std,0.303880
min,0.362203
25%,0.972095
50%,1.091370
75%,1.258650
max,1.636821


In [150]:
healthy.keys()

dict_keys([2, 8, 29, 33, 34, 37, 61, 71, 79, 80, 85, 87, 89, 91, 92, 95, 106, 108, 109, 110, 111, 114, 115, 116, 118, 124, 126])

In [1]:
def writeKEGGpdfcolors(outputDirectory, organism, superpathway, superpathwayDict, compound2color):
    """
    """
    index = []
    safeSuperpathway = makesafe(superpathway)
    for pw in superpathwayDict[superpathway]:
        countColor = {}
        for v in set(compound2color.values()):
            countColor[v] = set([])
        try:
            pathway = KGML_parser.read(kegg_get(organism + pw[0], "kgml"))
        except:
            print("Pathway", pw, "could not be downloaded: ignoring")
            continue
        canvas = KGMLCanvas(pathway)
        for k in pathway.entries:
            t = pathway.entries[k].type
            #pathway.entries[k].graphics[0].bgcolor = "#FFFFFF"   
            if t == "compound" and pathway.entries[k].is_reactant:
                compound = pathway.entries[k].name[4:]
                if compound in compound2color:
                    pathway.entries[k].graphics[0].bgcolor = compound2color[compound]
                    pathway.entries[k].graphics[0].fgcolor = compound2color[compound]
                    countColor[compound2color[compound]] |= set([compound])
                else:
                    pathway.entries[k].graphics[0].bgcolor = yellow
                    pathway.entries[k].graphics[0].bgcolor = yellow
        canvas.import_imagemap = True
        pdfName = organism + pw[0] + ".pdf"
        pathlib.Path(os.path.join(outputDirectory, organism, safeSuperpathway)).mkdir(parents=True, exist_ok=True)
        canvas.draw(os.path.join(outputDirectory, organism, safeSuperpathway, pdfName))
        index += [(pdfName, pw[1], len(countColor[valA]), len(countColor[valB]))]
    return index

In [49]:
pathway = KGML_parser.read(open(os.path.join(dataDirectory, organism, organism+"04510.xml"), "r"))
canvas = KGMLCanvas(pathway)

In [50]:
for k in pathway.entries:
    name = pathway.entries[k].name
    element = set2can(words2set(name))
    if element in element2id.keys():
        print(element, cc[element2id[element]])

['hsa:3688', 'hsa:3690', 'hsa:3691', 'hsa:3693', 'hsa:3694', 'hsa:3695', 'hsa:3696'] 14
['hsa:22801', 'hsa:3655', 'hsa:3672', 'hsa:3673', 'hsa:3674', 'hsa:3675', 'hsa:3676', 'hsa:3678', 'hsa:3679', 'hsa:3680', 'hsa:3685', 'hsa:8515', 'hsa:8516'] 13
['hsa:2316', 'hsa:2317', 'hsa:2318'] 116
['hsa:998'] 116
['cpd:C05981'] 54
['hsa:340156', 'hsa:4638', 'hsa:85366', 'hsa:91807'] 81
['hsa:103910', 'hsa:10398', 'hsa:10627', 'hsa:29895', 'hsa:4633', 'hsa:4636', 'hsa:58498', 'hsa:93408'] 21
['hsa:1729'] 33
['hsa:4659', 'hsa:4660', 'hsa:54776', 'hsa:5499', 'hsa:5500', 'hsa:5501'] 120
['hsa:6093', 'hsa:9475'] 33
['hsa:10000', 'hsa:207', 'hsa:208'] 41
['hsa:5170'] 42
['hsa:29780', 'hsa:55742', 'hsa:64098'] 99
['hsa:3611'] 15
['hsa:7414'] 100
['hsa:7408'] 63
['hsa:5728'] 110
['hsa:5290', 'hsa:5291', 'hsa:5293', 'hsa:5295', 'hsa:5296', 'hsa:8503'] 35
['hsa:7094', 'hsa:83660'] 11
['hsa:7791'] 116
['hsa:23396', 'hsa:8394', 'hsa:8395'] 33
['hsa:81', 'hsa:87'] 12
['hsa:5747'] 94
['hsa:824'] 70
['hsa:582

In [114]:
def columns2nodes(columns, node2columns, ignoreEmpty = True):
    """
        Returns a map from columns to the nodes where the column appears.
    """
    res = {} 
    for c in columns:
        s = set([])
        for no, co in node2columns.items():
            if c in co:
                s |= set([no])
        if len(s) > 0 or not ignoreEmpty:
            res[c] = s
    return res

c2n = columns2nodes(columns, node2columns)
usableColumns = list(c2n.keys())

In [115]:
def areRelated(column1, column2, column2nodes, node2color):
    nodes1 = column2nodes[column1]
    nodes2 = column2nodes[column2]
    colors1 = set([node2color[node] for node in nodes1])
    colors2 = set([node2color[node] for node in nodes2])
    return len(colors1 & colors2) > 0

In [121]:
genes.loc[:,"MYB"].values.tolist()

[5.701896,
 5.733379,
 5.67861,
 5.807889,
 5.697066,
 5.575801,
 6.042641,
 5.927202,
 5.545422,
 5.786408,
 5.95011,
 5.815601,
 6.012413,
 5.793511,
 5.677289,
 5.933953,
 5.947393,
 5.899629,
 5.804845,
 6.092096,
 5.745906,
 6.011995,
 5.825637,
 5.731103,
 5.582764,
 6.190875,
 6.344836,
 5.856002,
 5.713507,
 5.768836,
 5.985311,
 6.517211,
 6.108443,
 6.185022,
 5.65606,
 5.917612,
 5.835067,
 5.781578,
 5.544848,
 5.642407,
 5.81435,
 5.645506,
 5.881913,
 5.876419,
 5.987989,
 5.584053,
 5.692786,
 5.765418,
 5.750019,
 5.693265,
 5.830058,
 5.323685,
 5.832056,
 5.38001,
 5.618241,
 5.939519,
 6.120963,
 5.823152,
 5.869081,
 5.845005,
 5.974405,
 5.862833,
 6.207702,
 5.702713]

In [160]:
reltaus = []
unreltaus = []

for i in range(100000):
    gene1 = random.choice(usableColumns)
    gene2 = random.choice(usableColumns)
    expr1 = genes.loc[:,gene1].values.tolist()
    expr2 = genes.loc[:,gene2].values.tolist()
    tau = scipy.stats.kendalltau(expr1, expr2).statistic
    if areRelated(gene1, gene2, c2n, cc):
        reltaus += [tau]
    else:
        unreltaus += [tau]


In [161]:
print(pd.DataFrame(reltaus).describe())

                  0
count  10150.000000
mean       0.030742
std        0.201300
min       -0.563492
25%       -0.085317
50%        0.018849
75%        0.114300
max        1.000000


In [162]:
print(pd.DataFrame(unreltaus).describe())

                  0
count  89850.000000
mean       0.011626
std        0.164281
min       -0.627976
25%       -0.094246
50%        0.006944
75%        0.112103
max        0.740079


In [165]:
scipy.stats.ks_2samp(reltaus, unreltaus, alternative="less")

KstestResult(statistic=0.03111831158115197, pvalue=2.0866789228662175e-08, statistic_location=0.00546041352661954, statistic_sign=-1)

In [147]:
genes.loc[classification[genes["name"]]=="healthy"]

KeyError: "None of [Index(['GSM1574423', 'GSM1574424', 'GSM1574425', 'GSM1574426', 'GSM1574427',\n       'GSM1574428', 'GSM1574429', 'GSM1574430', 'GSM1574431', 'GSM1574432',\n       'GSM1574433', 'GSM1574434', 'GSM1574435', 'GSM1574436', 'GSM1574437',\n       'GSM1574438', 'GSM1574439', 'GSM1574440', 'GSM1574441', 'GSM1574442',\n       'GSM1574443', 'GSM1574444', 'GSM1574445', 'GSM1574446', 'GSM1574447',\n       'GSM1574448', 'GSM1574449', 'GSM1574450', 'GSM1574451', 'GSM1574452',\n       'GSM1574453', 'GSM1574454', 'GSM1574455', 'GSM1574456', 'GSM1574457',\n       'GSM1574458', 'GSM1574459', 'GSM1574460', 'GSM1574461', 'GSM1574462',\n       'GSM1574463', 'GSM1574464', 'GSM1574465', 'GSM1574466', 'GSM1574467',\n       'GSM1574468', 'GSM1574469', 'GSM1574470', 'GSM1574471', 'GSM1574472',\n       'GSM1574473', 'GSM1574474', 'GSM1574475', 'GSM1574476', 'GSM1574477',\n       'GSM1574478', 'GSM1574479', 'GSM1574480', 'GSM1574481', 'GSM1574482',\n       'GSM1574483', 'GSM1574484', 'GSM15744